#### This is a notebook about analyzing music files

In [ ]:
# Create Fourier transformation on the WAV file
# Good ideas from here: https://stackoverflow.com/questions/23377665/python-scipy-fft-wav-files
# http://myinspirationinformation.com/uncategorized/audio-signals-in-python/
# https://stackoverflow.com/questions/34742225/how-to-extract-data-from-a-wav-file-using-python-matplotlib-library
# https://stackoverflow.com/questions/2648151/python-frequency-detection

import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy.signal import blackman, hanning, hamming
import numpy as np
%matplotlib inline


# Size of each chunk for the FFT function to process
chunk_size = 4096

# Location of the song
files_location = "/Users/valentin/Documents/MusicEngine/wav/"     #"/Users/valentin/Documents/MusicEngine/wav/"
song_id = "TARGET_Biz_Amulet"                                        #"TARGET_Biz_Amulet"

# Returns the sample rate (in samples/sec) and data from a WAV file
fs, soundtrack_data = wavfile.read(files_location + song_id + ".wav")
audio_data = soundtrack_data.T[0]                      # this is a two channel soundtrack, get only one of the tracks
audio_data = audio_data[1:(44100*5+1)]

audio_data.dtype                                       # the data is stored as int16, i.e. the sound is 16 bit
soundtrack_length = len(soundtrack_data) / fs          # calculate length of soundtrack in seconds


# Plot the signal in the time domain
time = np.arange(0, float(soundtrack_data.shape[0]), 1) / fs

# Plot amplitude (or loudness) over time
plt.figure(figsize = (30, 20))
plt.plot(time, soundtrack_data, linewidth = 0.1, alpha = 2, color='#ff0000')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")


# Create Fourier transform - THIS NEEDS TO BE DONE IN CHUNKS OF 2048 or 4096
fourier_transform = fft(audio_data)   # Calculate fourier transform (complex numbers list)
window = hanning(round(len(audio_data)/chunk_size))
#fourier_transform = fft( NEED TO ADD THE DATA HERE * window)

d = int(len(fourier_transform) / 2)                 # Find the mid-point of FFT. I only need half of the fft list (the signal is symmetric)

# Data exploration
#print(fs)                                # Sampling rate
#print(data.T.shape)                      
#print(data.T[0].min(), data.T[0].max())  # min and max values for channel 1
#print(data.T[1].min(), data.T[1].max())  # min and max values for channel 2


In [ ]:
# The values in the data represent the amplitude of the wave (or the loudness of the audio)
# The energy of the audio can be described by the sum of the absolute amplitude.
# This will depend on the length of the audio, the sample rate and the volume of the audio
# A better metric is the Power which is energy per second

# Energy of music
energy = np.sum(audio_data.astype(float) ** 2)

# Power - energy per unit of time
power = 1.0 / (2 * (audio_data.size) + 1) * np.sum(audio_data.astype(float) ** 2) / fs

print ("Energy of music is: ", energy, "\nThe power of the music is: ", power)

In [ ]:
# Plot the FFT
#plt.yscale('log')
plt.figure(figsize = (30, 20))
plt.plot(abs(fourier_transform[:(d - 1)]), linewidth = 0.08, alpha = 3, color = '#ff0000') 
plt.xlabel("k")
plt.ylabel("Amplitude")


In [ ]:
# The Fourier Transform creates a imaginary. The symmetry of the complex Fourier transform is very important
# A real time domain signal corresponds to a frequency spectrum with an even real part, and an odd imaginary part (http://www.analog.com/media/en/technical-documentation/dsp-book/dsp_book_Ch31.pdf)
# We only need the real data solution, so we can grab the first half, then calculate the frequency and plot the frequency against a scaled amplitude
length_audio_data = len(audio_data)
fourier_transform_real_part = fourier_transform[0:int(length_audio_data / 2)]

# Scale by the number of observations so that the magnitude does not depend on the length
fourier_transform_real_part_scaled = fourier_transform_real_part / float(length_audio_data)

# Calculate the frequency at each point in Hz
frequency_of_song = np.arange(0, (length_audio_data / 2), 1.0) * (int(fs) / length_audio_data)

# Calculate power
power_db = 10 * np.log10(fourier_transform_real_part_scaled)

In [ ]:
#len(fourier_transform_real_part)
#fourier_transform_real_part.max(), fourier_transform_real_part.min()
print (fourier_transform_real_part_scaled.max(), fourier_transform_real_part_scaled.min())
print (frequency_of_song.max(), frequency_of_song.min())


In [ ]:
plt.figure(figsize = (30, 20))
plt.plot(frequency_of_song / 1000, power_db, color = '#ff0000', linewidth = 0.08)
plt.xlabel("Frequency (kHz)")
plt.ylabel("Power (dB)")

In [ ]:
# Plot a spectrogram
# Location of the song
files_location = "/Users/valentin/Documents/MusicEngine/wav/"     #"/Users/valentin/Documents/MusicEngine/wav/"
song_id = "SIOiqyC9vQE"                                     #"TARGET_Biz_Amulet"; Suzanita - 2kvAahQmWnc


# Returns the sample rate (in samples/sec) and data from a WAV file
fs, soundtrack_data = wavfile.read(files_location + song_id + ".wav")
audio_data = soundtrack_data.T                      # this is a two channel soundtrack, get only one of the tracks

plt.figure(2, figsize = (30, 20))
Pxx, freqs, bins, im = plt.specgram(audio_data,
                                    Fs = fs,
                                    NFFT = 1024,
                                    #window = window_hanning(),
                                    cmap = plt.get_cmap('autumn_r'))
#cbar = plt.colorbar(im)
#plt.xlabel('Time (s)')
#plt.ylabel('Frequency (Hz)')
#cbar.set_label('Intensity dB')
#plt.show()


In [ ]:
# Examine particular regions
np.where(freqs == 1000)
MHZ10 = Pxx[233,:]
plt.plot(bins, MHZ10, color = '#ff7f00')

In [ ]:
print(fourier_transform.shape, audio_data.shape)